In [1]:
from juliacall import Main as jl
jl.seval("""
         import Pkg
         ENV["JULIA_SSL_CA_ROOTS_PATH"]=""
         ENV["JULIA_PKG_USE_CLI_GIT"]=true
         Pkg.add(url=\"https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git\")
         """)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


    Updating git-repo `https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git`
   Resolving package versions...
    Updating `C:\Users\mking\.julia\environments\pyjuliapkg\Project.toml`
  [a1edf54d] ~ KLLS v1.0.0-DEV `https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git#main`
    Updating `C:\Users\mking\.julia\environments\pyjuliapkg\Manifest.toml`
  [a1edf54d] ~ KLLS v1.0.0-DEV `https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/MPF-Optimization-Laboratory/KLLS.jl.git#main`
Precompiling project...
  ✓ SciMLBase → SciMLBasePythonCallExt
  ✓ DiffEqBase → DiffEqBaseChainRulesCoreExt
  ✓ LinearSolve → LinearSolveEnzymeExt
  ✓ SimpleNonlinearSolve → SimpleNonlinearSolveChainRulesCoreExt
  ✓ NonlinearSolve
  ✓ KLLS
  6 dependencies successfully precompiled in 106 seconds. 194 already precompiled.


In [104]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np

# For MNIST data download
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

import mnist_vae.vae as vae
plt.style.use('dark_background')
def to_im(x):
    return x.cpu().detach().numpy().reshape((28, 28))

%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
path = Path()
data_dir = path

test_root = os.path.join(data_dir, "data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing above, no need to re-run

In [105]:
jl.seval("""
         using KLLS
         solve = KLLS.solve!
         rescale = KLLS.scale!
         maximize = KLLS.maximize!
         regularize = KLLS.regularize!
         dObj = KLLS.dObj!
         dGrad = KLLS.dGrad!
         """);

In [108]:

net = vae.VAE()
net.load_state_dict(torch.load('mnist_vae/weights/vae_epoch_25.pth'))


<All keys matched successfully>

In [109]:
def objective(q,g):
    value = jl.dObj(kl,q)
    gradient_value = jl.dGrad(kl,q,g)
    return [value, gradient_value]

def scalar_obj(q):
    value = jl.dObj(kl,q)
    return value


In [110]:
#{ Dataset
#test_data = datasets.MNIST(
#    root="data",
##   download=True,
#    transform=ToTensor()
# )
#dataloader = torch.utils.data.DataLoader(test_data, batch_size=1,
#                                             shuffle=False)

# commented out for now; but code for dcgan

In [112]:
#test_img = next(iter(dataloader))

#test_img_for_plot = test_img[0][0].detach().numpy()
#plt.imshow(np.reshape(test_img_for_plot,[28,28]), cmap='gray');

# as above

tensor([[[ 1.5900, -0.2048,  1.4300],
         [-0.2498,  3.0586,  3.7770],
         [-0.6684,  3.3708,  5.8614]],

        [[11.6963, 16.6248,  6.2208],
         [-2.7520, 16.6404, -0.2493],
         [-0.0570, -3.5455, -0.2500]],

        [[ 3.4558,  1.1330,  2.0457],
         [ 0.9616,  0.5694, -0.3113],
         [ 0.3502,  2.6659,  0.2291]],

        ...,

        [[ 4.6920,  4.0211,  6.4823],
         [ 3.9899,  5.4358,  2.7810],
         [ 3.4497,  0.7824,  3.6690]],

        [[ 0.5546, -0.5012, -0.5232],
         [-1.0774, -1.9827, -1.6974],
         [-0.4337, -2.0954, -1.0226]],

        [[-0.5217, -1.6745, -1.9092],
         [-0.8658, -1.7318, -1.6779],
         [-1.2985, -2.4019, -1.5029]]], grad_fn=<SelectBackward0>)
